# docker练习场 笔记

## 题目
分阶段提交容器镜像完成以下3个任务（分数依次占 30/30/40），根据评分系统的分数返回验证任务的完成情况。

- 输出Hello world
- 计算 /tcdata/num_list.csv中一列数字的总和
- 在/tcdata/num_list.csv文件中寻找最大的10个数，从大到小生成一个List

num_list.csv文件中只有一列不为负的整数，其中存在重复值，示例如下：
> 102
6
11
11

生成入口脚本`run.sh`，放置于镜像工作目录。运行后生成结果`result.json`放置于工作目录（与`run.sh`同目录），评分系统将根据`result.json`进行打分。json文件如下所示：
```
{  
    "Q1":"Hello world", 
    "Q2":sum值, 
    "Q3":[top10_list] 
}
```

### 构建镜像并推送

为简化构建镜像的难度，天池已准备了常用的Python基础镜像，可直接拉取使用
```
docker pull registry.cn-shanghai.aliyuncs.com/tcc-public/python:3
```
#### 1. 准备所需文件
新建一个文件夹（例如tianchi_submit_demo）用于存放这次任务镜像所需的文件，文件夹中如下：
> Dockerfile  hello_world.py   result.json   run.sh

其中`hello_world.py`中是自己的代码部分：

`Dockerfile`配置文件参考，`Dockerfile`是固定名称，首字母大写。`Dockerfile`中命令皆大写：
```
# Base Images
## 从天池基础镜像构建
FROM registry.cn-shanghai.aliyuncs.com/tcc-public/python:3

## 把当前文件夹里的文件构建到镜像的根目录下
ADD . /

## 指定默认工作目录为根目录（需要把run.sh和生成的结果文件都放在该文件夹下，提交后才能运行）
WORKDIR /

## 镜像启动后统一执行 sh run.sh
CMD ["sh", "run.sh"]
```
`run.sh`参考:
```
python hello_world.py
```
#### 2. 构建镜像并推送  -- 服务器上直接操作

执行
```
docker build -t registry.cn-shenzhen.aliyuncs.com/test_for_tianchi/test_for_tianchi_submit:1.0 .
```

注意：`registry.~~~` 是上面创建仓库的公网地址，用自己仓库地址替换。地址后面的：1.0为自己指定的版本号，用于区分每次build的镜像。最后的.是构建镜像的路径，不可以省掉。

构建完成后可先验证是否正常运行，正常运行后再进行推送。

CPU镜像：`docker run your_image sh run.sh`

GPU镜像：`nvidia-docker run your_image sh run.sh`

推送到镜像仓库 `docker push registry.cn-shenzhen.aliyuncs.com/test_for_tianchi/test_for_tianchi_submit:1.0`

如果这步出错，可能是因为没有登录，按照仓库里描述操作登录即可。

## 1. 开通容器镜像服务（推荐阿里云）
